<a href="https://colab.research.google.com/github/mephisto83/MagicWandPy/blob/main/MagicWand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
#Start by connecting gdrive into the google colab

#from google.colab import drive

#drive.mount('/content/gdrive')

In [121]:
source_folder = "F:\\split\\split_files" #@param {type:"string"} "cfg/training/yolov7_pothole-tiny.yaml"
output_folder = "F:\\split\\magic_wanded_files\\" #@param {type:"string"} "cfg/training/yolov7_pothole-tiny.yaml"


In [122]:
import os

def ensure_folder_exists(folder_path):
    """
    Ensures that a folder exists at the specified path.
    If the path points to a file, creates the necessary parent directory to ensure that just the folder exists.
    """
    if os.path.isfile(folder_path):
        folder_path = os.path.dirname(folder_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder at {folder_path}")
    else:
        print(f"Folder already exists at {folder_path}")



In [123]:
import os
from PIL import Image
import numpy as np

def save_numpy_as_png_with_alpha(image, file_name):
    # If the image has only 3 channels, add an alpha channel with full opacity
    if image.shape[2] == 3:
        alpha = np.ones((image.shape[0], image.shape[1], 1), dtype=np.uint8) * 255
        image = np.concatenate((image, alpha), axis=2)

    # Create a PIL image from the numpy array with mode 'RGBA'
    image_rgba = Image.fromarray(image, mode='RGBA')

    # Save the image as a PNG file
    image_rgba.save(file_name)

def compare_folder_files(folder_path1, folder_path2):
    # Get the list of files in each folder
    files1 = os.listdir(folder_path1)
    files2 = os.listdir(folder_path2)

    # Filter out any subdirectories in each folder
    files1 = [f for f in files1 if os.path.isfile(os.path.join(folder_path1, f))]
    files2 = [f for f in files2 if os.path.isfile(os.path.join(folder_path2, f))]

    # Find the files in folder1 that don't have matching names in folder2
    unique_files = [os.path.join(folder_path1, f) for f in files1 if f not in files2]

    # Return the list of unique file paths
    return unique_files

def get_file_name(file_path):
    # Get the base name of the file (i.e. the last part of the path)
    base_name = os.path.basename(file_path)

    # Return the base name, which includes the extension
    return base_name

In [124]:
def get_color_difference(color1, color2):
    """
    Calculates the difference between two colors using the CIE76 formula.

    Parameters:
    color1 (tuple): The first color as an RGB tuple.
    color2 (tuple): The second color as an RGB tuple.

    Returns:
    float: The color difference between the two colors as a float.
    """
    # Convert the RGB colors to the CIE XYZ color space
    xyz1 = rgb_to_xyz(color1)
    xyz2 = rgb_to_xyz(color2)

    # Convert the CIE XYZ colors to the CIE L*a*b* color space
    lab1 = xyz_to_lab(xyz1)
    lab2 = xyz_to_lab(xyz2)

    # Calculate the Euclidean distance between the two colors in Lab space
    dE = math.sqrt((lab1[0] - lab2[0]) ** 2 + (lab1[1] - lab2[1]) ** 2 + (lab1[2] - lab2[2]) ** 2)

    return dE

def rgb_to_xyz(rgb):
    """
    Converts an RGB color to the CIE XYZ color space.

    Parameters:
    rgb (tuple): The RGB color as a tuple of three integers.

    Returns:
    tuple: The XYZ color as a tuple of three floats.
    """
    # Normalize the RGB values to be between 0 and 1
    r = rgb[0] / 255.0
    g = rgb[1] / 255.0
    b = rgb[2] / 255.0

    # Apply the RGB to XYZ matrix transformation
    x = r * 0.4124 + g * 0.3576 + b * 0.1805
    y = r * 0.2126 + g * 0.7152 + b * 0.0722
    z = r * 0.0193 + g * 0.1192 + b * 0.9505

    return (x, y, z)

def xyz_to_lab(xyz):
    """
    Converts a color from the CIE XYZ color space to the CIE L*a*b* color space.

    Parameters:
    xyz (tuple): The XYZ color as a tuple of three floats.

    Returns:
    tuple: The Lab color as a tuple of three floats.
    """
    # Normalize the XYZ values to the reference white point
    x_n = 0.95047
    y_n = 1.0
    z_n = 1.08883
    x = xyz[0] / x_n
    y = xyz[1] / y_n
    z = xyz[2] / z_n

    # Apply the XYZ to Lab matrix transformation
    t = 0.008856
    delta = 6.0 / 29.0
    f = lambda x: x ** (1.0 / 3.0) if x > t else 3 * delta ** 2 * (x - t / 3.0)
    l = 116.0 * f(y) - 16.0
    a = 500.0 * (f(x) - f(y))
    b = 200.0 * (f(y) - f(z))

    return (l, a, b)




In [125]:
from PIL import Image, ImageChops

def magic_wand(file_path, tolerance=10, file_output_path=None):
    # Open the image file
    image = Image.open(file_path)

    # Find a color close to white in each of the four corners
    reference_colors = []
    for x in [0, image.width - 1]:
        for y in [0, image.height - 1]:
            quadrant = image.crop((x, y, x + image.width // 2, y + image.height // 2))
            reference_colors.append(quadrant.getpixel((0, 0)))

    # Create a blank image with the same size as the original image
    result = Image.new('RGBA', image.size, (0, 0, 0, 0))

    # Iterate over each reference color and mask
    for reference_color in reference_colors:
        # Create a blank mask for this reference color
        mask = Image.new('RGBA', image.size, (0, 0, 0, 0))

        # Use a queue to keep track of pixels to check
        queue = [(0, 0), (0, image.width - 1), (image.width - 1, 0) , (image.width - 1, image.width - 1)]

        # Iterate over the queue until it's empty
        while queue:
            # Pop the next pixel from the queue
            x, y = queue.pop(0)

            # Skip if the pixel has already been processed
            if mask.getpixel((x, y))[3] != 0:
                continue

            # Get the color of the current pixel
            current_color = image.getpixel((x, y))

            # Calculate the difference in color between the current pixel and the reference color
            # color_difference = sum([abs(reference_color[k] - current_color[k]) for k in range(3)])
            color_difference = get_color_difference(reference_color, current_color)
            # If the difference in color is within the tolerance, set the pixel in the mask to white
            if color_difference <= tolerance:
                mask.putpixel((x, y), (255, 255, 255, 255))

                # Add neighboring pixels to the queue
                if x > 0:
                    queue.append((x - 1, y))
                if y > 0:
                    queue.append((x, y - 1))
                if x < image.size[0] - 1:
                    queue.append((x + 1, y))
                if y < image.size[1] - 1:
                    queue.append((x, y + 1))

        # Invert the mask to select the contiguous area
        mask = ImageChops.invert(mask)

        # Paste the original image onto the result image using the mask
        result.paste(image, (0, 0), mask)

    # Save the result image as a PNG if file_output_path is provided
    if file_output_path:
        result.save(file_output_path)

    return result


In [126]:
files = compare_folder_files(source_folder, output_folder)
for i in range(len(files)): # 
  filename = files[i]
  tolerances = [20 ,5]
  for j in range(len(tolerances)):
    tol = tolerances[j]
    ensure_folder_exists(os.path.join(output_folder, f"{tol}"))
    output_file_path = os.path.join(output_folder, f"{tol}", get_file_name(filename))
    magic_wand(filename, tol, output_file_path)


Created folder at F:\split\magic_wanded_files\20
Created folder at F:\split\magic_wanded_files\17
Created folder at F:\split\magic_wanded_files\15
Folder already exists at F:\split\magic_wanded_files\10
Created folder at F:\split\magic_wanded_files\5
Folder already exists at F:\split\magic_wanded_files\20
Folder already exists at F:\split\magic_wanded_files\17
Folder already exists at F:\split\magic_wanded_files\15
Folder already exists at F:\split\magic_wanded_files\10
Folder already exists at F:\split\magic_wanded_files\5
Folder already exists at F:\split\magic_wanded_files\20
Folder already exists at F:\split\magic_wanded_files\17
Folder already exists at F:\split\magic_wanded_files\15
Folder already exists at F:\split\magic_wanded_files\10
Folder already exists at F:\split\magic_wanded_files\5
Folder already exists at F:\split\magic_wanded_files\20
Folder already exists at F:\split\magic_wanded_files\17
Folder already exists at F:\split\magic_wanded_files\15
Folder already exists a

KeyboardInterrupt: 